In [ ]:
%matplotlib ipympl
import time
from pymongo import MongoClient 
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from datetime import datetime
from bson.objectid import ObjectId
from matplotlib.animation import FuncAnimation
from collections import Counter

In [ ]:
hostip = "192.168.0.160"
client = MongoClient(
        host=f'{hostip}',
        port=27017
    )
db = client["awas_db"]
violation_col = db["violation"]

print(violation_col.count_documents({}))

for doc in violation_col.find().limit(5):
    print(doc)

In [ ]:
from dateutil import parser as dt_parser

all_timestamps = [] 
processed_counts = {} 

def fetch_latest_data():
    global processed_counts
    new_timestamps = []

    # Fetch all documents 
    for doc in collection.find():
        
        doc_id = doc["_id"]
        violations = doc.get("violations", [])
        total = len(violations)
        print(total)
    
        already_seen = processed_counts.get(doc_id, 0)
        
         # Get only new violations
        new_violations = violations[already_seen:]
        processed_counts[doc_id] = total  # update count
        
        for v in new_violations:
            ts_str = v.get("timestamp_start")
            new_timestamps.apppend(ts_str)
            
                
    print(f"Fetched {len(new_timestamps)} new timestamps")
    return new_timestamps


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
# line, = ax.plot([], [], 'bo-')

def update(frame):
    
    global all_timestamps
    
    # Fetch new timestamps
    new_data = fetch_latest_data()
        
    all_timestamps.extend(new_data)

    if not all_timestamps:
        print("no data")
        return

    # Count and sort timestamps
    counter = Counter(all_timestamps)
    sorted_times = sorted(counter)
    counts = [counter[t] for t in sorted_times]

    ax.clear()
    ax.plot(sorted_times, counts, 'bo-', label="Violations")
    ax.set_title("Violations per Arrival Time")
    ax.set_xlabel("Arrival Time")
    ax.set_ylabel("Number of Violations")

ani = FuncAnimation(fig, update, interval=1000, cache_frame_data=False)  # every 60 sec
plt.tight_layout()
plt.show()



In [ ]:
from pymongo import MongoClient
from dateutil import parser as dt_parser

client = MongoClient(host=hostip, port=27017)
db = client["awas_db"]
violation_col = db["violation"]

print("Total documents:", violation_col.count_documents({}))

cursor = violation_col.find()

timestamps = []

for doc in cursor:
    violations = doc.get("violations", [])

    for v in violations:
        ts_str = v.get("timestamp_start")
        try:
            ts = dt_parser.parse(ts_str)
            rounded_ts = ts.replace(second=0, microsecond=0)
            timestamps.append(rounded_ts)
        except Exception as e:
            print(f"    Error parsing '{ts_str}': {e}")

print(f"\n✅ Total timestamps parsed: {len(timestamps)}")

# from collections import Counter
# import matplotlib.pyplot as plt

# # Your parsed timestamps list
# # timestamps = [...] ← already built from previous step

# counter = Counter(timestamps)
# sorted_times = sorted(counter)
# counts = [counter[t] for t in sorted_times]

# # print("Time slots:")
# # for t, c in zip(sorted_times, counts):
# #     print(f"{t}: {c} violations")

# plt.figure(figsize=(10, 5))
# plt.plot(sorted_times, counts, 'bo-', label='Violations')
# plt.xticks(rotation=45)
# plt.xlabel("Arrival Time (rounded to minute)")
# plt.ylabel("Number of Violations")
# plt.title("Violations per Arrival Time")
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# Each frame adds more timestamps
frame_counter = 0
all_data = []

fig, ax = plt.subplots(figsize=(10, 5))

def update(frame):
    global frame_counter, timestamps

    # Add one timestamp per frame for testing
    if frame_counter < len(timestamps):
        all_data.append(timestamps[frame_counter])
        frame_counter += 1

    if not all_data:
        return

    counter = Counter(all_data)
    sorted_times = sorted(counter)
    counts = [counter[t] for t in sorted_times]

    ax.clear()
    ax.plot(sorted_times, counts, 'bo-')
    ax.set_title("Animated Test Plot")
    ax.set_xlabel("Arrival Time")
    ax.set_ylabel("Violations")
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True)

ani = FuncAnimation(fig, update, interval=1000)  # update every second for test
plt.tight_layout()
plt.show()



In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from collections import Counter
from datetime import datetime, timedelta

# Simulate streaming data every frame
simulated_data = [
    datetime(2024, 1, 1, 8, 0),
    datetime(2024, 1, 1, 8, 1),
    datetime(2024, 1, 1, 8, 1),
    datetime(2024, 1, 1, 8, 2),
    datetime(2024, 1, 1, 8, 2),
    datetime(2024, 1, 1, 8, 2),
]

# Each frame adds more timestamps
frame_counter = 0
all_data = []

fig, ax = plt.subplots(figsize=(10, 5))

def update(frame):
    global frame_counter, all_data

    # Add one timestamp per frame for testing
    if frame_counter < len(simulated_data):
        all_data.append(simulated_data[frame_counter])
        frame_counter += 1

    if not all_data:
        return

    counter = Counter(all_data)
    sorted_times = sorted(counter)
    counts = [counter[t] for t in sorted_times]

    ax.clear()
    ax.plot(sorted_times, counts, 'bo-')
    ax.set_title("Animated Test Plot")
    ax.set_xlabel("Arrival Time")
    ax.set_ylabel("Violations")
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True)

ani = FuncAnimation(fig, update, interval=1000)  # update every second for test
plt.tight_layout()
plt.show()


In [ ]:
from kafka3 import KafkaConsumer
import json
from dateutil import parser
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

hostip = "192.168.0.160"

consumer = KafkaConsumer(
    "violations",
    bootstrap_servers=f"{hostip}:9092",
    auto_offset_reset="latest",
    value_deserializer=lambda m: json.loads(m.decode("utf-8")),
    enable_auto_commit=True
)

timestamps = []

def poll_kafka():
    new_ts = []
    for msg in consumer.poll(timeout_ms=1000).values():
        for record in msg:
            print(record.value["timestamp_start"])
            ts = parser.parse(record.value["timestamp_start"])
            new_ts.append(ts.replace(second=0, microsecond=0))
    return new_ts


In [ ]:
fig, ax = plt.subplots()
all_timestamps = []

def update(frame):
    new_data = poll_kafka()
    all_timestamps.extend(new_data)

    if not all_timestamps:
        return

    counter = Counter(all_timestamps)
    sorted_times = sorted(counter)
    counts = [counter[t] for t in sorted_times]

    ax.clear()
    ax.plot(sorted_times, counts, 'bo-')
    ax.set_title("Live Violations from Kafka")
    ax.set_xlabel("Arrival Time")
    ax.set_ylabel("Violations")

ani = FuncAnimation(fig, update, interval=1000)  # every second
plt.tight_layout()
plt.show()
